In [ ]:
import os
from os.path import join
import sys
sys.path.append("../")

from DataAnalyzer import DataAnalyzer
from time import perf_counter

DATA_ROOT = "/home/guest/work/Datasets"

paths = {
    "picai_labels_wg" : "picai_labels_all/picai_labels-main/anatomical_delineations/whole_gland/AI/Guerbet23",
    "picai_labels_zonal" : "picai_labels_all/picai_labels-main/anatomical_delineations/zonal_pz_tz/AI/Yuan23",
    "picai_folds" : "picai_folds/"
}

analyzer = DataAnalyzer(DATA_ROOT)


> Every patient case will at least have three imaging sequences: axial T2W, axial DWI and axial ADC scans (i.e. files ending in _t2w.mha, _hbv.mha, _adc.mha). Additionally, they can also have either, both or none of these optional imaging sequences: sagittal and coronal T2W scans (i.e. files ending in _sag.mha, _cor.mha here). 

Vemos que algunos pacientes tienen imágenes adicionales

In [4]:
# pth = os.path.join(paths["picai_folds"])
# folds = analyzer.get_dirs(pth)

# for fold in folds:
#     rel = os.path.join(pth, fold)
#     dirs = analyzer.get_dirs(rel)
#     incomplete_dirs = []
    
#     for dir in dirs:
#         # check if they all have 5 files
#         rel2 = os.path.join(rel, dir)
        
#         files = analyzer.get_files(rel2)
        
#         if len(list(files)) != 5:
#             print(f"Directory {dir} has {len(list(files))} files, expected 5.")
#             incomplete_dirs.append(dir)

#     print(f"Number of non-matching directories: {len(incomplete_dirs)}")

#     print("Total file count excluding the previous non-matching: ", (len(list(dirs)) - len(incomplete_dirs) ) * 5)



In [ ]:
import random

# pick a random folder and set the name of a file inside as the prefix
d = "picai_folds/picai_images_fold0"
dirs = list(analyzer.get_dirs(d))
random_dir = random.choice(dirs)
files_in_dir = analyzer.get_files(join(d, random_dir), ".*_t2w.mha$")

name = list(files_in_dir)[0] # we have to do this because get files is a generator
f = join(d, random_dir, name)
analyzer.show_image(f)


['10961', '10851', '10437', '10760', '10615', '11086', '10935', '11313', '10647', '10719', '11021', '10913', '11249', '11392', '10962', '10934', '11353', '10395', '10502', '10156', '10001', '10902', '10908', '11220', '10813', '11003', '10675', '10509', '11020', '10526', '10359', '11176', '11151', '10071', '10129', '11425', '10868', '10761', '11400', '10685', '10198', '11161', '10571', '11137', '11197', '10791', '11159', '10263', '11171', '11166', '10624', '11294', '10521', '11120', '10847', '10772', '11474', '10302', '10921', '11248', '11084', '10065', '10807', '10873', '10195', '11196', '10524', '11111', '11079', '11266', '10029', '11221', '10862', '10599', '11383', '10006', '11285', '10933', '10439', '10085', '10438', '11157', '10595', '11467', '11341', '10078', '10981', '11173', '11234', '11389', '11473', '10834', '11307', '10083', '10282', '10942', '10257', '10392', '10879', '11106', '10062', '10459', '10572', '10747', '11334', '10400', '10736', '10027', '10939', '11035', '10541', 

SystemExit: 1

In [ ]:
nii = name.split("_t2w")[0] + ".nii.gz"
f = join(paths['picai_labels_zonal'], nii)
analyzer.show_image(f)

Let's look for possible empty masks, since some of the lesion data was missing

In [ ]:
# non_empty_count, empty_masks = analyzer.count_and_find_non_empty_masks(paths['picai_labels_zonal'])
# print(f"{non_empty_count} out of {non_empty_count+len(empty_masks)} masks are non-empty (contain a segmentation).")
# print("Empty mask files:", empty_masks)

Now, we use the function the analyzer to analyze the metadata.

In [ ]:
# use this regex to filter the files
analyzer.regex = "(.*_t2w.mha$)|(.*_sag.mha$)|(.*_cor.mha$)"
res = analyzer.collect_metadata_to_dataframe("picai_folds/picai_images_fold0/10189")
print(res)

start = perf_counter()
df = analyzer.collect_metadata_from_subdirs("picai_folds/picai_images_fold0")
print(df)
print(perf_counter() - start, "seconds")

In [ ]:
# for all the rows with vendor SIEMENS, get number of unique mri_names
siemens_mri_names = df[df['vendor'] == 'SIEMENS']['mri_name'].value_counts()

import matplotlib.pyplot as plt
siemens_mri_names.plot(kind='bar')
plt.title("""Number of unique MRI names for SIEMENS 
          (total count: {})""".format(siemens_mri_names.sum()))
plt.xlabel("MRI Name")
plt.ylabel("Count")
plt.show()

In [ ]:
# for all the rows with vendor Philips Medical Systems, get number of unique mri_names
phl_mri_names = df[df['vendor'] == 'Philips Medical Systems']['mri_name'].value_counts()

# plot in a bar graph
import matplotlib.pyplot as plt
phl_mri_names.plot(kind='bar')
plt.title("""Number of unique MRI names for Philips Medical Systems 
          (total count: {})""".format(phl_mri_names.sum()))
plt.xlabel("MRI Name")
plt.ylabel("Count")
plt.show()

In [ ]:
orientations = {"RAI" : "axial", "ASL" : "sagital", "RSA" : "coronal"}

for orientation, value in orientations.items():
    orientation_df = df[df['orientation'] == orientation]
    # get the dim_size for each orientation and plot it
    dim_sizes = orientation_df['dim_size'].value_counts()

    dim_sizes.plot(kind='bar')
    plt.title(f"Dim sizes for orientation {value} (total count: {dim_sizes.sum()})")
    plt.xlabel("Dim Size")
    plt.ylabel("Count")
    plt.show()